In [75]:
import importlib
import lab2
import copy
import numpy as np
from cv2 import imread, imwrite, imshow, waitKey, COLOR_BGR2RGB, cvtColor, vconcat, hconcat, putText, \
    FONT_HERSHEY_SIMPLEX, resize
import time

#from diffusion import take_params, get_probs, fill_graph_edges, diffusion_alg, labeling
# importlib.reload(lab2)

Read the image as float array, rescale it and set the parameters:

In [76]:
filename = "field4.jpg"
img1 = imread(filename)
scale = [256, 256]
img1_res = resize(img1, scale).astype(np.double)
n_iterations = 100
eps = 0.1

Zero iteration:

- take parameters of normal distribution from sky and ground on image;
- create an array of probabilities of class 0 and class 1 for each pixel;
- create start graph with edges depends on epsilon and probabilities;

In [77]:
start_time = time.time()

mean0, mean1, cov0, cov1 = lab2.take_params(img1_res)
PROBS = lab2.get_probs(img1_res, mean0, mean1, cov0, cov1)
graph = lab2.fill_graph_edges(PROBS)

zero_iter_time = time.time() - start_time
print("Zero iteration time:  %s seconds; " % zero_iter_time)

Zero iteration time:  0.27491140365600586 seconds; 


Some start graph edges:

In [78]:
print(graph[5, 2] , "\n")
print(graph[-2, -5])

[ -18.38909484  -64.92618995  -63.98835303 -114.91989731] 

[-880.96444233 -428.2904542  -466.6682609   -18.38872192]


Diffusion algorithm:

In [79]:
start_time = time.time()

new_graph = lab2.diffusion_alg(n_iterations, graph)

alg_time = time.time() - start_time
print("Diffusion time: %s seconds;" % alg_time)
print("time per diffusion iteration: ", alg_time / n_iterations)

Diffusion time: 1.319575548171997 seconds;
time per diffusion iteration:  0.013195755481719971


Same edges after diffusion:

In [80]:
print(new_graph[5, 2])
print(new_graph[-2, -5])
# np.savetxt('0.txt', new_graph[...,0 ],fmt='%.4f', delimiter='_')
# np.savetxt('1.txt', new_graph[...,1 ],fmt='%.4f',  delimiter='_')
# np.savetxt('2.txt', new_graph[...,2 ],fmt='%.4f',  delimiter='_')
# np.savetxt('3.txt', new_graph[...,3 ],fmt='%.4f',  delimiter='_')


[ 0.09427583  0.09427583  0.09427583 -4.30017333]
[-4.6617421  -0.26729294 -0.26729294 -0.26729294]


Now we need to pick labeling for graph:

In [81]:
start_time = time.time()

graph_for_label = copy.deepcopy(new_graph)
labeling_graph, final_labels = lab2.labeling(graph_for_label)

alg_time = time.time() - start_time
print("Labeling time: %s seconds;" % alg_time)
# print("time per Labeling iteration: ", alg_time / n_iterations)

labeling...
Start epsilon:  4.394579820011432
Deletions algorithm has ended
For epsilon =  2.197289910005716 : looking...
Deletions algorithm has ended
For epsilon =  1.098644955002858 : looking...
Deletions algorithm has ended
For epsilon =  0.549322477501429 : looking...
Deletions algorithm has ended
For epsilon =  0.2746612387507145 : looking...
Deletions algorithm has ended
For epsilon =  0.13733061937535726 : looking...
Deletions algorithm has ended
Final labeling founded;
 Final epsilon =  0.13733061937535726
Labeling time: 3.298940658569336 seconds;


In [82]:
print(final_labels)

result = np.zeros_like(img1_res)
result[..., 0] = final_labels * 255
result[..., 1] = final_labels * 255
result[result[..., 0] == 0] = [0, 0, 255]
result[result[..., 1] == 0] = [0, 0, 255]
result2 = hconcat([(img1_res//2 + result //2), img1_res])
imwrite('result2.png', result2)

[[0 0 0 ... 1 0 0]
 [0 0 1 ... 0 0 1]
 [0 0 1 ... 0 0 0]
 ...
 [1 0 1 ... 1 1 0]
 [0 0 1 ... 0 1 0]
 [1 0 0 ... 1 0 0]]


True